In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [14]:
premise_file = "../Resources/LicensePremise.csv"
premise_csv = pd.read_csv(premise_file)
premise_csv

,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2
...,...,...,...,...,...
1858,1311940,H CITY DISCOUNT STORE INC,NaN,2018-07-13T00:00:00,14
1859,1311941,YONY P DELI CORP,NaN,2018-07-13T00:00:00,14
1860,2213432,JOSEPH SCHLEIMER,NaN,2018-07-13T00:00:00,12
1861,1311943,DAIRY SHACK LLC,NaN,2018-07-13T00:00:00,10


In [15]:
county_file = "../Resources/CountyLicenseCount.csv"
county_csv = pd.read_csv(county_file)
county_csv


,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9
...,...,...,...,...
57,57,WASHINGTON,15,5
58,58,WAYNE,52,9
59,59,WESTCHESTER,20,73
60,60,WYOMING,56,2


### Transform premise DataFrame

In [42]:
premise_df_new = premise_csv[['License Serial Number', 'Premises Name', 'County ID Code']].copy()
premise_df_new = premise_df_new.rename(columns={"License Serial Number":"id","Premises Name":"premise_name","County ID Code":"county_id"})
premise_df_new

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2
...,...,...,...
1858,1311940,H CITY DISCOUNT STORE INC,14
1859,1311941,YONY P DELI CORP,14
1860,2213432,JOSEPH SCHLEIMER,12
1861,1311943,DAIRY SHACK LLC,10


### Transform county DataFrame

In [40]:
county_df_new = county_csv[['ID','County Name (Licensee)','License Count', 'County ID Code']].copy()
county_df_new = county_df_new.rename(columns={"ID":"id","County Name (Licensee)":"county_name","License Count":"license_count","County ID Code":"county_id"})
county_df_new

,id,county_name,license_count,county_id
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41
...,...,...,...,...
57,57,WASHINGTON,5,15
58,58,WAYNE,9,52
59,59,WESTCHESTER,73,20
60,60,WYOMING,2,56


### Create database connection

In [31]:
connection_string = 'postgres:root123@localhost:5432/postgres'
engine = create_engine(f'postgresql://{connection_string}')

['county', 'premise']

In [32]:
# Confirm tables
engine.table_names()

### Load DataFrames into database

In [51]:
premise_df_new.to_sql(name='premise', con=engine, if_exists='replace', index=False)

In [47]:
county_df_new.to_sql(name='county', con=engine, if_exists='replace', index=False)

# Read SQL

In [54]:
pd.read_sql_query('select * from premise', con=engine).head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2


In [48]:
pd.read_sql_query('select * from county', con=engine).head()

,id,county_name,license_count,county_id
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41


In [57]:
sql_join = r"""SELECT premise.id,premise.premise_name,county.county_name 
                FROM premise, county
                WHERE premise.county_id = county.county_id;"""
        
res = pd.read_sql(sql_join, con=engine)

In [58]:
res

,id,premise_name,county_name
0,1311660,CANA ARRIBA GROCERY NO 2 INC,BRONX
1,1310023,JOHANA GROCERY & DELI CORP,BRONX
2,1310024,TKO BEVERAGES LLC,QUEENS
3,1310024,TKO BEVERAGES LLC,QUEENS
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NEW YORK
...,...,...,...
1858,1311940,H CITY DISCOUNT STORE INC,KINGS
1859,1311941,YONY P DELI CORP,KINGS
1860,2213432,JOSEPH SCHLEIMER,ROCKLAND
1861,1311943,DAIRY SHACK LLC,SUFFOLK
